In [1]:
import logging

import numpy

import cicada.additive
import cicada.communicator

logging.basicConfig(level=logging.INFO)

@cicada.communicator.NNGCommunicator.run(world_size=2)
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator)

    # Player 0 will provide a secret dividend.
    a = numpy.array(37) if communicator.rank == 0 else None
    # All players know the divisor.
    b = numpy.array(7)
    
    log.info(f"Dividend: {a}", src=0)
    log.info(f"Divisor: {b}", src=1)

    # Compute the quotient
    a_share = protocol.share(src=0, secret=protocol.encoder.encode(a), shape=())
    quotient_share = protocol.untruncated_private_public_divide(a_share, b)
    quotient_share = protocol.truncate(quotient_share)
    quotient = protocol.encoder.decode(protocol.reveal(quotient_share))
    log.info(f"Player {communicator.rank} quotient: {quotient}")

main();

INFO:cicada.communicator.nng:Player 0 rendezvous with tcp://127.0.0.1:50900 from tcp://127.0.0.1:50900.
INFO:cicada.communicator.nng:Player 1 rendezvous with tcp://127.0.0.1:50900 from tcp://127.0.0.1:50901.
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator ready.
INFO:root:Dividend: 37
INFO:root:Divisor: 7
INFO:root:Player 0 quotient: 5.285552978515625
INFO:root:Player 1 quotient: 5.285552978515625
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator freed.
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator freed.
INFO:cicada.communicator.nng:Player 0 returned: None
INFO:cicada.communicator.nng:Player 1 returned: None


As you can see, the result revealed to both players is a close approximation to the expected value.